In [ ]:
import subprocess
import csv
import time
from datetime import datetime, timedelta
import speedtest

# -------- CONFIG --------
PING_TARGETS = [
    "10.36.17.152",
    "10.36.17.127",
    "10.36.22.120",
    "10.16.25.15",
]  # Add/modify as needed
INTERVAL_SECONDS = 60 * 5  # Collect data every 5 minutes
TOTAL_RUNTIME_HOURS = 15  # Total runtime in hours (5 minutes for testing)
OUTPUT_FILE = "network_log1.csv"
# ------------------------


def ping_host(host):
    """
    Send 4 pings to host and parse summary:
    Returns: sent, received, lost, min_rtt, avg_rtt, max_rtt
    """
    try:
        # -c 4 : send 4 packets
        result = subprocess.run(
            ["ping", "-c", "20", host], capture_output=True, text=True
        )

        output = result.stdout

        # Extract packet stats
        # Example: "4 packets transmitted, 4 received, 0% packet loss"
        packet_line = [
            line for line in output.split("\n") if "packets transmitted" in line
        ][0]
        parts = packet_line.split(",")

        sent = int(parts[0].split()[0])
        received = int(parts[1].split()[0])
        lost = sent - received

        # Extract RTT stats
        # Example: "rtt min/avg/max/mdev = 14.123/16.234/20.456/1.234 ms"
        rtt_line = [line for line in output.split("\n") if "rtt min" in line][0]
        rtt_values = rtt_line.split("=")[1].split("/")

        min_rtt = float(rtt_values[0])
        avg_rtt = float(rtt_values[1])
        max_rtt = float(rtt_values[2])

        return sent, received, lost, min_rtt, avg_rtt, max_rtt

    except Exception as e:
        return 0, 0, 0, None, None, None


def measure_speed():
    """
    Returns: ping, download_Mbps, upload_Mbps
    """
    try:
        st = speedtest.Speedtest()
        st.get_best_server()
        ping = st.results.ping
        down = st.download() / 1e6  # convert to Mbps
        up = st.upload() / 1e6
        return ping, down, up
    except:
        return None, None, None


def main():
    end_time = datetime.now() + timedelta(hours=TOTAL_RUNTIME_HOURS)

    # Write CSV header
    with open(OUTPUT_FILE, "w", newline="") as f:
        writer = csv.writer(f)
        header = ["timestamp", "speedtest_ping", "download_Mbps", "upload_Mbps"]
        # Add ping stats for each host
        for host in PING_TARGETS:
            header += [
                f"{host}_sent",
                f"{host}_received",
                f"{host}_lost",
                f"{host}_min_rtt",
                f"{host}_avg_rtt",
                f"{host}_max_rtt",
            ]
        writer.writerow(header)

    print("✅ Monitoring started. Logging to:", OUTPUT_FILE)

    # Monitoring loop
    while datetime.now() < end_time:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Speed test
        st_ping, st_down, st_up = measure_speed()

        row = [timestamp, st_ping, st_down, st_up]

        # Ping results for each IP
        for host in PING_TARGETS:
            sent, recv, lost, min_rtt, avg_rtt, max_rtt = ping_host(host)
            row += [sent, recv, lost, min_rtt, avg_rtt, max_rtt]

        # Append to CSV
        with open(OUTPUT_FILE, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(row)

        print(f"[{timestamp}] Logged one entry.")
        time.sleep(INTERVAL_SECONDS)

    print("✅ 15 hours completed. Monitoring stopped.")


if __name__ == "__main__":
    main()

✅ Monitoring started. Logging to: network_log1.csv
[2025-11-12 10:48:54] Logged one entry.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

CSV_FILE = "network_log.csv"  # Same filename used by your main script

df = pd.read_csv(CSV_FILE)
df["timestamp"] = pd.to_datetime(df["timestamp"])

# -----------------------------
# 1. Download Speed
# -----------------------------
plt.figure()
plt.plot(df["timestamp"], df["download_Mbps"])
plt.title("Download Speed (Mbps)")
plt.xlabel("Time")
plt.ylabel("Mbps")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# -----------------------------
# 2. Upload Speed
# -----------------------------
plt.figure()
plt.plot(df["timestamp"], df["upload_Mbps"])
plt.title("Upload Speed (Mbps)")
plt.xlabel("Time")
plt.ylabel("Mbps")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# -----------------------------
# 3. Speedtest Ping
# -----------------------------
plt.figure()
plt.plot(df["timestamp"], df["speedtest_ping"])
plt.title("Speedtest Ping (ms)")
plt.xlabel("Time")
plt.ylabel("Ping (ms)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# -----------------------------
# 4. Ping Latency for Each IP
# -----------------------------
PING_TARGETS = [
    "10.36.17.152",
    "10.36.17.127",
    "10.36.22.120",
    "10.16.25.15",
]

for host in PING_TARGETS:
    col = f"{host}_avg_rtt"

    if col in df:
        plt.figure()
        plt.plot(df["timestamp"], df[col])
        plt.title(f"Average Ping to {host} (ms)")
        plt.xlabel("Time")
        plt.ylabel("Latency (ms)")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

# -----------------------------
# 5. Packet Loss for Each IP
# -----------------------------
for host in PING_TARGETS:
    sent_col = f"{host}_sent"
    recv_col = f"{host}_received"

    if sent_col in df and recv_col in df:
        loss = df[sent_col] - df[recv_col]

        plt.figure()
        plt.plot(df["timestamp"], loss)
        plt.title(f"Packet Loss to {host}")
        plt.xlabel("Time")
        plt.ylabel("Lost Packets")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter  # <-- correct import

# Input CSV file
CSV_FILE = "network_log.csv"

# Read CSV
df = pd.read_csv(CSV_FILE)
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Create figure
fig = plt.figure(figsize=(10, 5))

# Plot download, upload, and ping
plt.plot(
    df["timestamp"],
    df["download_Mbps"],
    label="Download (Mbps)",
    color="red",
    linewidth=1.8,
)
plt.plot(
    df["timestamp"],
    df["upload_Mbps"],
    label="Upload (Mbps)",
    color="tab:orange",
    linewidth=1.8,
)
ax1 = plt.gca()
ax1.set_ylabel("Speed (Mbps)")
# plt.legend()

ax2 = plt.gca().twinx()  # Create a second y-axis
ax2.plot(
    df["timestamp"],
    df["speedtest_ping"],
    label="Ping (ms)",
    color="tab:green",
    linewidth=1.8,
)
ax2.set_ylim(0, 5000)
ax2.set_ylabel("Ping (ms)", color="tab:green")


# Format x-axis to show time like "4 PM"
def hour_formatter(x, pos=None):
    dt = mdates.num2date(x)
    return dt.strftime("%I %p").lstrip("0")  # removes leading 0 on Windows


# plt.gca().xaxis.set_major_formatter(FuncFormatter(hour_formatter))
# plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=1))

# Labels and legend
plt.title("Internet Speed and Ping Over Time")
plt.xlabel("Time of Day")
# plt.legend(["a", "b", "c"])
fig.legend()
plt.grid(True, linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

In [8]:
import matplotlib.pyplot as plt

plt.plot(df["timestamp"], df["download_Mbps"])

In [ ]:
df["timestamp"], df["timestamp1"]

(0      11-11-2025 18:11
 1      11-11-2025 18:16
 2      11-11-2025 18:22
 3      11-11-2025 18:28
 4      11-11-2025 18:34
              ...       
 142    12-11-2025 08:48
 143    12-11-2025 08:55
 144    12-11-2025 09:01
 145    12-11-2025 09:07
 146    12-11-2025 10:48
 Name: timestamp, Length: 147, dtype: object,
 0     2025-11-11 18:11:00
 1     2025-11-11 18:16:00
 2     2025-11-11 18:22:00
 3     2025-11-11 18:28:00
 4     2025-11-11 18:34:00
               ...        
 142   2025-12-11 08:48:00
 143   2025-12-11 08:55:00
 144   2025-12-11 09:01:00
 145   2025-12-11 09:07:00
 146   2025-12-11 10:48:00
 Name: timestamp1, Length: 147, dtype: datetime64[ns])